In this notebook we process the glaciers of the Antarctic Peninsula. This includes a large range so we will not cover
the full range.

In [1]:
# Import script to embed geolocation.
from IPython.display import HTML
import datetime

import sys
sys.path.extend(['/Users/gertmulder/software/rippl_main'])

from rippl.orbit_geometry.read_write_shapes import ReadWriteShapes
from rippl.SAR_sensors.sentinel.sentinel_download import DownloadSentinel
from rippl.processing_templates.general_sentinel_1 import GeneralPipelines

glacier_shape = [[-62.84179687499999, -66.89559561140706],
            [-59.45800781249999, -64.68031793418314],
            [-59.94140624999999, -63.84066844285507],
            [-64.072265625, -64.99793920061401],
            [-66.357421875, -66.40795547978848],
            [-67.1484375, -67.32292427995345],
            [-67.587890625, -68.57644086491784],
            [-66.97265625, -69.24057862839376],
            [-64.4677734375, -70.3335333600625],
            [-63.58886718749999, -71.60828252210263],
            [-63.6328125, -73.15043991163012],
            [-65.654296875, -74.09197431391085],
            [-66.62109375, -75.05035357407698],
            [-62.57812500000001, -75.20824498631067],
            [-60.29296874999999, -73.92246884621463],
            [-59.501953125, -73.09941313082074],
            [-59.58984374999999, -72.19524581118353],
            [-60.380859375, -70.80136623397622],
            [-61.34765625, -69.62651016802958],
            [-62.22656249999999, -68.73638345287264],
            [-63.6328125, -67.89208614070753],
            [-62.84179687499999, -66.89559561140706]]
study_area = ReadWriteShapes()
study_area(glacier_shape)

geojson = study_area.shape

# Try to do the same by creating a shapefile with QGIS, geojson online or a .kml file in google earth.
# study_area.read_kml(kml_path)
# study_area.read_geo_json(geojson_path)
# study_area.read_shapefile(shapefile_path)

The next step in the processing is selection of the right track, as the Sentinel-1 satellite will cover the area on both
ascending and descending tracks. However, to do so, we will have to define the start and end date of our data search, 
because the satellite is not always acquiring data. 
Next search will give a small oversight of the available tracks during our period of interest. In our case we will 
search one week before and one week after the earthquake.

Watch out with extending the search window, this can result in a lot of images. Note that every downloaded file will take approximately 4GB of disk space!

In [2]:
# First we search using a time window

# Track and data type of Sentinel data
mode = 'IW'
product_type = 'SLC'
polarisation = 'HH'

# First we check using a time window
start_date = datetime.datetime(year=2017, month=10, day=1)
end_date = datetime.datetime(year=2018, month=10, day=1)

#find_track = DownloadSentinel(start_date=start_date, end_date=end_date, shape=study_area.shape, sensor_mode=mode,
#                              polarisation=polarisation)
#find_track.sentinel_search_ASF()
#find_track.summarize_search_results(plot_cartopy=False, buffer=2)

After selection of the right track we can start the actual download of the images. In our case we use track 38.

This will download our data automatically to our radar database. Additionally, it will download the precise orbit files.
These files are created within a few weeks after the data acquisition and define the satellite orbit within a few cm
accuracy. These orbits are necessary to accurately define the positions of the radar pixels on the ground later on
in the processing.

In [3]:
# Track and data type of Sentinel data
mode = 'IW'
product_type = 'SLC'
polarisation = 'HH'

from rippl.processing_templates.general_sentinel_1 import GeneralPipelines

# Create the list of the 4 different stacks.
track_no = 38
stack_name = 'glaciers_Antarctic_Peninsula'
# For every track we have to select a master date. This is based on the search results earlier.
# Choose the date with the lowest coverage to create an image with only the overlapping parts.
master_date = datetime.datetime(year=2018, month=4, day=26)

# Number of processes for parallel processing. Make sure that for every process at least 2GB of RAM is available
no_processes = 4

s1_processing = GeneralPipelines(processes=no_processes)
#s1_processing.download_sentinel_data(start_date=start_date, end_date=end_date, track=track_no,
#                                           polarisation=polarisation, shapefile=study_area.shape, data=True, source='ASF')
#s1_processing.create_sentinel_stack(start_date=start_date, end_date=end_date, master_date=master_date, cores=no_processes,
#                                          track=track_no,stack_name=stack_name, polarisation=polarisation,
#                                          shapefile=study_area.shape, mode=mode, product_type=product_type)

# Finally load the stack itself. If you want to skip the download step later, run this line before other steps!
s1_processing.read_stack(start_date=start_date, end_date=end_date, stack_name=stack_name)

Reading stack from folder /media/gert/Processing disk/Sentinel-1/glaciers_Antarctic_Peninsula


To define the location of the radar pixels on the ground we need the terrain elevation. Although it is possible to 
derive terrain elevation from InSAR data, our used Sentinel-1 dataset is not suitable for this purpose. Therefore, we
download data from an external source to create a digital elevation model (DEM). In our case we use TanDEM-X data.

However, to find the elevation of the SAR data grid, we have to resample the data to the radar grid first to make it
usable. This is done in the next steps.

In [4]:
# Some basic settings for DEM creation.
dem_buffer = 0.1        # Buffer around radar image where DEM data is downloaded
dem_rounding = 0.1      # Rounding of DEM size in degrees
dem_type = 'TanDEM-X'   # DEM type of data we download (SRTM1, SRTM3 and TanDEM-X are supported)

# Define both the coordinate system of the full radar image and imported DEM
s1_processing.create_radar_coordinates()
s1_processing.create_dem_coordinates(dem_type=dem_type, lon_resolution=3)

# Download external DEM
s1_processing.download_external_dem(dem_type=dem_type, buffer=dem_buffer, rounding=dem_rounding, 
                                    n_processes=no_processes, lon_resolution=3)

Using the obtained elevation model the exact location of the radar pixels in cartesian (X,Y,Z) and geographic (Lat/Lon)
can be derived. This is only done for the master or reference image. This process is referred to as geocoding.

In [8]:
# Geocoding of image
s1_processing.geocoding()

Reading stack from folder /media/gert/Processing disk/Sentinel-1/glaciers_Antarctic_Peninsula
Processing pipeline block 1 out of 11
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already finished
Processing pipeline block 2 out of 11
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already finished
Processing pipeline block 3 out of 11
Skipping processing. Process already finished
Skipping p

The information from the geocoding can directly be used to find the location of the master grid pixels in the slave
grid images. This process is called coregistration. Because the orbits are not exactly the same with every satellite 
overpass but differ hundreds to a few thousand meters every overpass, the grids are slightly shifted with respect to 
each other. These shift are referred to as the spatial baseline of the images. To correctly overlay the master and slave
images the software coregisters and resamples to the master grid.

To do so the following steps are done:
1. Coregistration of slave to master image
2. Deramping the doppler effects due to TOPs mode of Sentinel-1 satellite
3. Resampling of slave image
4. Reramping resampled slave image.

Due to the different orbits of the master and slave image, the phase of the radar signal is also shifted. We do not 
know the exact shift of the two image, but using the geometry of the two images we can estimate the shift of the phase
between different pixels. Often this shift is split in two contributions:
1. The flat earth phase. This phase is the shift in the case the earth was a perfect ellipsoid
2. The topographic phase. This is the phase shift due to the topography on the ground.
In our processing these two corrections are done in one go.

In [9]:
# Next step applies resampling and phase correction in one step.
# Polarisation
s1_processing.geometric_coregistration_resampling(polarisation=polarisation, output_phase_correction=True)

Reading stack from folder /media/gert/Processing disk/Sentinel-1/glaciers_Antarctic_Peninsula
Processing pipeline block 1 out of 633
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already finished
Processing pipeline block 2 out of 633
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already finished
Processing pipeline block 3 out of 633
Skipping processing. Process already finished
Skippin

Now we can create calibrated amplitudes, interferograms and coherences.

In [5]:
for dist in [50, 100, 200]:

    # The actual creation of the calibrated amplitude images
    s1_processing.create_ml_coordinates(standard_type='oblique_mercator', dx=dist, dy=dist, buffer=0, rounding=0)
    s1_processing.prepare_multilooking_grid(polarisation)
    s1_processing.create_calibrated_amplitude_multilooked(polarisation)
    s1_processing.create_output_tiffs_amplitude()

    s1_processing.create_ifg_network(temporal_baseline=15)
    s1_processing.create_interferogram_multilooked(polarisation)
    s1_processing.create_coherence_multilooked(polarisation)

    # Create output geotiffs
    s1_processing.create_output_tiffs_coherence_ifg()

    # Create lat/lon/incidence angle/DEM for multilooked grid.
    s1_processing.create_geometry_mulitlooked()
    s1_processing.create_output_tiffs_geometry()

Reading stack from folder /media/gert/Processing disk/Sentinel-1/glaciers_Antarctic_Peninsula
Using predefined grid size for concatenation of slices
Concatenated dataset already exists. If you want to overwrite set overwrite to True
Processing pipeline block 1 out of 1
Reading stack from folder /media/gert/Processing disk/Sentinel-1/glaciers_Antarctic_Peninsula
Processing pipeline block 1 out of 8
Processing pipeline block 2 out of 8
Processing pipeline block 3 out of 8
Processing pipeline block 4 out of 8
Processing pipeline block 5 out of 8
Processing pipeline block 6 out of 8
Processing pipeline block 7 out of 8
Processing pipeline block 8 out of 8
Processing 0 out of 67
Processing 1 out of 67
Processing 2 out of 67
Processing 3 out of 67
Processing 4 out of 67
Processing 5 out of 67
Processing 6 out of 67
Processing 7 out of 67
Processing 8 out of 67
Processing 9 out of 67
Processing 10 out of 67
Processing 11 out of 67
Processing 12 out of 67
Processing 13 out of 67
Processing 14 

/home/gert/Software/rippl_main/rippl/meta_data/image_data.py:454: RuntimeWarning: divide by zero encountered in log
  data.GetRasterBand(1).WriteArray(np.log(np.abs(self.memory['data'])))
/home/gert/Software/rippl_main/rippl/meta_data/image_data.py:454: RuntimeWarning: divide by zero encountered in log
  data.GetRasterBand(1).WriteArray(np.log(np.abs(self.memory['data'])))
/home/gert/Software/rippl_main/rippl/meta_data/image_data.py:454: RuntimeWarning: divide by zero encountered in log
  data.GetRasterBand(1).WriteArray(np.log(np.abs(self.memory['data'])))
